In [1]:
import tensorflow as tf
import os
zip_file = tf.keras.utils.get_file(origin="https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip", fname="cats_vs_dogs.zip", extract=True, cache_subdir=os.getcwd())
base_dir, _ = os.path.splitext(zip_file)
print(base_dir)

824901632/824894548 [==============================] - 5s 0us/step
/content/cats_vs_dogs


In [2]:
import numpy as np
import matplotlib.pyplot as plt
#import tqdm as tqdm 
from tqdm import tqdm
import cv2
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision
import torch.nn.functional as F


REBUILD_DATA = True

In [3]:
os.listdir('PetImages')

['Dog', 'Cat']

In [4]:
class cats_vs_dogs():
  IMG_SIZE = 50
  cat_images = '/content/PetImages/Cat'
  dog_images = '/content/PetImages/Dog'
  labels = {'Cat' : 0, 'Dog' : 1}
  training_data = []
  cat_count = 0
  dog_count = 0

  def make_dataset(self):
    for i in self.labels:
      folder_path = os.path.join('/content/PetImages', i)
      for j in os.listdir(folder_path):
        try:
          file_path = os.path.join(folder_path, j)
          img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
          img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
          self.training_data.append([np.array(img), np.eye(2)[0 if i == 'Cat' else 1]])
          if i == 'Cat':
            self.cat_count += 1
          elif i == 'Dog':
            self.dog_count += 1
        except Exception as e:
          pass 

    np.random.shuffle(self.training_data)
    np.save('training_data.npy', self.training_data)
    print('Cats:', self.cat_count)
    print('Dog:', self.dog_count)

if REBUILD_DATA:
  catsvdogs = cats_vs_dogs()
  catsvdogs.make_dataset()
	



Cats: 12476
Dog: 12470


In [5]:
training_data = np.load('/content/training_data.npy', allow_pickle=True)

In [6]:
print(training_data[1])

[array([[26, 25, 27, ..., 34, 31, 33],
       [31, 33, 33, ..., 37, 35, 36],
       [40, 40, 36, ..., 37, 39, 39],
       ...,
       [75, 84, 82, ..., 25, 21, 22],
       [73, 79, 76, ..., 20, 19, 18],
       [67, 72, 65, ..., 17, 17, 17]], dtype=uint8)
 array([1., 0.])]


In [20]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size=5, padding=2, stride=1) 
    self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5, padding = 2, stride = 1)
    self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 5, padding = 2, stride = 1)
    #self.fc1 = nn.Flatten((6,6,128), 6*6*128)
    self.fc1 = nn.Flatten()
    self.fc2 = nn.Linear(6*6*128, 512)
    self.out = nn.Linear(512, 2)
  


  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2)) # Input(50,50,1) , output(25,25,32)
    x = F.max_pool2d(F.relu(self.conv2(x)), (2,2)) # Input(25,25,32), Output(12,12,64)
    x = F.max_pool2d(F.relu(self.conv3(x)), (2,2)) # Input(12,12,64), Output(6,6,128)
    #print(x[0].shape)
    x = self.fc1(x) # Input(6,6,128), Output(6*6*128)
    x = F.relu(self.fc2(x)) # Input(6*6*128), Output(512)
    x = F.softmax(self.out(x), dim=1) # Input(512), Output(2)
    return x 

net = Net()

In [21]:
net

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Flatten(start_dim=1, end_dim=-1)
  (fc2): Linear(in_features=4608, out_features=512, bias=True)
  (out): Linear(in_features=512, out_features=2, bias=True)
)

In [22]:
from sklearn.model_selection import train_test_split
x = torch.Tensor([i[0] for i in training_data])
x = x/255.0
y = torch.Tensor([i[1] for i in training_data])

train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.15, shuffle=True)

In [23]:
output = net(x[1].view(-1, 1, 50, 50))
output

tensor([[0.4925, 0.5075]], grad_fn=<SoftmaxBackward>)

In [24]:
train_y.shape

torch.Size([21204, 2])

In [25]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

net = Net().to(device)

In [29]:
import torch.optim as optim 
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
epochs=50
batch_size=100
for epoch in tqdm(range(epochs)):
  count = 0 
  for i in range(0, len(train_x), batch_size):
    xs, ys = train_x[i: i+batch_size], train_y[i: i+batch_size]
    net.zero_grad()
    output = net(xs.view(-1, 1, 50, 50).to(device))
    loss = loss_fn(output, ys.to(device))
    loss.backward()
    optimizer.step()
  print(f'epoch : {epoch}, LOSS : {loss}')
  #print('LOSS: ', loss)








  '''for xs, ys in zip(train_x, train_y):
    #print(xs,ys)
    net.zero_grad()
    output = net(xs.view(-1, 1, 50, 50))
    loss = loss_fn(ys, output)
    loss.backward()
    optimizer.step()
    print(f'epoch {epoch}, iteration {count}')
    count += 1
    if count == 2000:
      break
  print('LOSS : ', loss)'''
    


  2%|▏         | 1/50 [00:02<02:20,  2.87s/it]

epoch : 0, LOSS : 0.002539336681365967



  4%|▍         | 2/50 [00:05<02:16,  2.85s/it]

epoch : 1, LOSS : 0.008569440804421902



  6%|▌         | 3/50 [00:08<02:12,  2.83s/it]

...

epoch : 47, LOSS : 3.7179474776660726e-11



 98%|█████████▊| 49/50 [02:14<00:02,  2.74s/it]

epoch : 48, LOSS : 7.826168439351022e-05



100%|██████████| 50/50 [02:17<00:00,  2.74s/it]

epoch : 49, LOSS : 2.7686797299253385e-10


In [30]:
correct = 0.0
total = 0.0
with torch.no_grad():
  for i in tqdm(range(len(val_x))):
    pred = torch.argmax(net(val_x[i].view(-1, 1, 50, 50).to(device)))
    real = torch.argmax(val_y[i].to(device))
    if pred == real:
      correct += 1
    total += 1

print('ACCURACY : ', correct/total)  


100%|██████████| 3742/3742 [00:03<00:00, 1175.09it/s]

ACCURACY :  0.780331373597007
